# SAKI Homework 4


## Setup

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
import os
os.chdir("/content/gdrive/MyDrive/flair")

In [4]:
!pip install pymdptoolbox

  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-cp37-none-any.whl size=25657 sha256=10568718763b3312296a640ca1b4fac64d1634925312ffe746dbdb2e7da01a31
  Stored in directory: /root/.cache/pip/wheels/87/a9/a8/40c4e252c02e590737265742425cdd0365fafcc162441a9527
Successfully built pymdptoolbox


In [5]:
import random
import mdptoolbox
import numpy as np

def read_file(filename):
  actions = []
  with open(filename, "r") as file:
    for line in file:
      actions.append(line.strip().split("\t"))
  return actions

trainings_data = read_file("warehousetraining2x2.txt")
test_data = read_file("warehouseorder2x2.txt")

## Data Exploration and Validation

First we verify that the test and trainings data is valid, that a 2x2 warehouse is large enough to fit all items and that all restore operations can be fulfilled.

Next we calculate the probabilities of each action.

In [6]:
def analyze_data_set(data):
  print("Actions: "+ str(len(data)))

  action_probabilities = { "store red": 0, "store blue": 0, "store white": 0, \
                           "restore red": 0, "restore blue": 0, "restore white": 0}
  max_size = 0
  warehouse = []
  for [action, item] in data:
    if action == "store":
      warehouse.append(item)
    elif action == "restore":
      warehouse.remove(item)
    else:
      raise Exception("invalid action")
    
    max_size = max(max_size, len(warehouse))
    action_probabilities[action+" "+item] += 1
  
  for key, value in action_probabilities.items():
    action_probabilities[key] /= len(data)
  
  print("max warehouse capacity: " + str(max_size))

  print("action probabilities:")
  for key, value in action_probabilities.items():
    print("\t", key, value)
  
  return action_probabilities

print("Trainings Data")
training_action_probabilities = analyze_data_set(trainings_data)
print("\nTest Data")
test_action_probabilities = analyze_data_set(test_data)

Trainings Data
Actions: 8177
max warehouse capacity: 4
action probabilities:
	 store red 0.25241531123884065
	 store blue 0.12168276874159227
	 store white 0.12596306713953773
	 restore red 0.25241531123884065
	 restore blue 0.12168276874159227
	 restore white 0.12584077289959644

Test Data
Actions: 65
max warehouse capacity: 4
action probabilities:
	 store red 0.24615384615384617
	 store blue 0.12307692307692308
	 store white 0.15384615384615385
	 restore red 0.23076923076923078
	 restore blue 0.12307692307692308
	 restore white 0.12307692307692308


## Markov Decision Process

The warehouse consists of a 2x2 grid. Each field can either be empty or hold one of three items. A robot can store items in and restore items from the grid. The robot moves only horizontal and vertical and has to enter/exit the warehouse using the field (0|0).

The 2x2 warehouse is modelled as a 4 element long list. Fields are indentified using a numeric index from 0-3. The field that the robot uses to enter/exit the warehouse has the index 0.



In [7]:
# Warehouse
# 0 | 1
# --|--
# 2 | 3

# distance between origin (field next to 0|0) and position
def distance(position):
  if position == 0:
    return 1
  elif position == 1 or position == 2:
    return 2
  elif position == 3:
    return 3
  else:
    raise Exception("position out of bounds")

# returns all locations that hold the specified item (use None to find free fields)
def search(warehouse, item):
  locations = []
  index = -1
  while True:
    try:
      index = warehouse.index(item, index+1)
      locations.append(index)
    except:
      break
  return locations

# tests if the  action can be fulfilled with in current state of the warehouse
def is_fulfillable(warehouse, action):
  action, item = action
  if action == "store":
    return len(search(warehouse, None)) > 0
  elif action == "restore":
    return len(search(warehouse, item)) > 0

# returns the new warehouse after storing the item
def apply_store(warehouse, location, item):
  new_warehouse = warehouse.copy()
  new_warehouse[location] = item
  return new_warehouse

# returns the new warehouse after restoring the item
def apply_restore(warehouse, location):
  new_warehouse = warehouse.copy()
  new_warehouse[location] = None
  return new_warehouse

The warehouse has 4 fields, each field can either be in one of 4 states (free, red, blue, white). The robot can receive one of 6 actions (store/restore red/blue/white). This makes a total of `4^4*6=1536` states.

The transition matrix contains probabilities for each field and each state to state combination. The reward matrix contains the distance to the selected field as a penalty or a penalty of -1000 for invalid actions.

In [12]:
field_count = 2*2
field_states = [None, "red", "blue", "white"]
actions = [("store", "red"), ("store", "blue"), ("store", "white"), ("restore", "red"), ("restore", "blue"), ("restore", "white")]

# the transition matrix has to be full of probabilities
# -> all probabilities have to sum up to 1
def make_stochastic(matrix):
  for i, row in enumerate(matrix):
    row_sum = sum(row)
    if row_sum == 0:
      matrix[i,i] = 1 # [i,i] is invalid with a large penalty
    else:
      for j, value in enumerate(row):
        matrix[i,j] = value/row_sum
  return matrix

def all_states():
  for x1 in field_states:
    for x2 in field_states:
      for x3 in field_states:
        for x4 in field_states:
          for action in actions:
            yield ([x1, x2, x3, x4], action)

state_count = len(field_states) ** field_count * len(actions)
all_transitions = []
rewards = np.zeros((state_count, field_count))
rewards.fill(-1000) # all invalid moves have a large penalty by default
for field in range(field_count):
  transitions = np.zeros((state_count, state_count))

  for i, (current_warehouse, (current_action, current_action_item)) in enumerate(all_states()):
    # calculate the resulting warehouse of the current action and field
    if current_action == "store":
      possible_locations = search(current_warehouse, None)
      resulting_warehouse = apply_store(current_warehouse, field, current_action_item) if field in possible_locations else None
    elif current_action == "restore":
      possible_locations = search(current_warehouse, current_action_item)
      resulting_warehouse = apply_restore(current_warehouse, field) if field in possible_locations else None

    for j, (next_warehouse, _) in enumerate(all_states()):
      # give reward if the next state is a valid successor state
      if next_warehouse == resulting_warehouse:
        action = current_action+" "+current_action_item
        transitions[i,j] = training_action_probabilities[action]
        rewards[i,field] = -distance(field)

  transitions = make_stochastic(transitions)
  all_transitions.append(transitions)

print("State count: " + str(state_count))
print("Transitions shape: 6x " + str(all_transitions[0].shape))
print("Rewards shape: " + str(rewards.shape))


State count: 1536
Transitions shape: 6x (1536, 1536)
Rewards shape: (1536, 4)


The transition and reward matrices are used to generate policies using the Markov Decision Process Framework. One policy is generated using PolicyIteration and one using ValueIteration.

In [13]:
mdpresultPolicy = mdptoolbox.mdp.PolicyIteration(all_transitions, rewards, 0.999)
mdpresultPolicy.run()
print(mdpresultPolicy.policy)
print(mdpresultPolicy.V)
print(mdpresultPolicy.iter)


(0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 3, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 3, 1, 0, 0, 0, 

In [14]:
mdpresultValue = mdptoolbox.mdp.ValueIteration(all_transitions, rewards, 0.999)
mdpresultValue.run()
print(mdpresultValue.policy)
print(mdpresultValue.V)
print(mdpresultValue.iter)


(0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 3, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 3, 1, 0, 0, 0, 

## Evaluation

To evaluate the policies generated using the Markov Decision Process the test dataset is used to simulate an order and the total distance traveled by the robot is calculated.

The results are compared with four handcrafted policies later on.

In [15]:
def evaluate_strategy(strategy, actions):
  warehouse = [None, None, None, None]
  position = 0
  travel_distance = 0

  for [action, item] in actions:
    field = strategy(warehouse, action, item)
    travel_distance += 2*distance(field) # there and back
    if action == "store":
      if warehouse[field] != None:
        raise Exception("can't store in warehouse")
      warehouse[field] = item
    elif action == "restore":
      if warehouse[field] != item:
        raise Exception("can't restore from warehouse")
      warehouse[field] = None
  
  return travel_distance


In [16]:
def get_state_index(warehouse, action, item):
  for i, (state_warehouse, (state_action, state_action_item)) in enumerate(all_states()):
    if warehouse == state_warehouse and action == state_action and item == state_action_item:
      return i
  raise Exception("invalid state")

def policy_strategy(policy):
  def _strategy(warehouse, action, item):
    index = get_state_index(warehouse, action, item)
    return policy.policy[index]
  return _strategy

policy_iteration_strategy_distance = evaluate_strategy(policy_strategy(mdpresultPolicy), test_data)
print("Strategy: Policy Iteration")
print("distance traveled: " + str(policy_iteration_strategy_distance))

value_iteration_strategy_distance = evaluate_strategy(policy_strategy(mdpresultValue), test_data)
print("\nStrategy: Value Iteration")
print("distance traveled: " + str(value_iteration_strategy_distance))


Strategy: Policy Iteration
distance traveled: 228

Strategy: Value Iteration
distance traveled: 228


To put these results into perspective they are compared with four other, handwritten strategies:

### Stategy 1: Store at the closest field, restore from the closest field

This strategy always chooses a valid field that has the minimal distance to the origin. The warehouse is filled and cleared from front to back.

In [17]:
def choose_closest_strategy(warehouse, action, item):
  if action == "restore":
    locations = search(warehouse, item)
    locations.sort(key=distance)
    return locations[0]
  elif action == "store":
    locations = search(warehouse, None)
    locations.sort(key=distance)
    return locations[0]

choose_closest_strategy_distance = evaluate_strategy(choose_closest_strategy, test_data)
print("Strategy: choose closest field")
print("distance traveled: " + str(choose_closest_strategy_distance))


Strategy: choose closest field
distance traveled: 228


### Strategy 2: Store at the farthest field, restore from the closest field

This strategy chooses the field with the largest distance from the origin to store items and restores from the closest field. The warehouse is filled back to front and cleared front to back.

In [18]:
def store_farthest_strategy(warehouse, action, item):
  if action == "restore":
    locations = search(warehouse, item)
    locations.sort(key=distance)
    return locations[0]
  elif action == "store":
    locations = search(warehouse, None)
    locations.sort(key=distance, reverse=True)
    return locations[0]

store_farthest_strategy_distance = evaluate_strategy(store_farthest_strategy, test_data)
print("Strategy: store on farthest field, restore from closest field")
print("distance traveled: " + str(store_farthest_strategy_distance))


Strategy: store on farthest field, restore from closest field
distance traveled: 274


### Strategy 3: Random stores, closest restores

This strategy places items at random but chooses the closest field for restoring items.

As this strategy relies on random numbers it is evaluated 100 times with different seeds and averaged to keep the results reproducible without cherry-picking one seed.

In [19]:
def store_random_strategy(warehouse, action, item):
  if action == "restore":
    locations = search(warehouse, item)
    locations.sort(key=distance)
    return locations[0]
  elif action == "store":
    locations = search(warehouse, None)
    return random.choice(locations)

print("Strategy: store on random field, restore from closest field")
travel_distances = []
for i in range(0, 100):
  random.seed(i)
  travel_distances.append(evaluate_strategy(store_random_strategy, test_data))
store_random_strategy_distance = sum(travel_distances) / len(travel_distances)
print("distance traveled: " + str(store_random_strategy_distance))


Strategy: store on random field, restore from closest field
distance traveled: 253.92


### Strategy 4: Random stores, random restores

This strategy places items at random and also restores items at random.

In [20]:
def choose_random_strategy(warehouse, action, item):
  if action == "restore":
    locations = search(warehouse, item)
    return random.choice(locations)
  elif action == "store":
    locations = search(warehouse, None)
    return random.choice(locations)

print("Strategy: store on random field, restore from random field")
travel_distances = []
for i in range(0, 100):
  random.seed(i)
  travel_distances.append(evaluate_strategy(choose_random_strategy, test_data))
choose_random_strategy_distance = sum(travel_distances) / len(travel_distances)
print("distance traveled: " + str(choose_random_strategy_distance))


Strategy: store on random field, restore from random field
distance traveled: 259.42


## Conclusion


In [21]:
import pandas as pd
pd.DataFrame.from_dict({
    "Policy Iteration": [policy_iteration_strategy_distance],
    "Value Iteration": [value_iteration_strategy_distance],
    "Store Closest, Restore Closest": [choose_closest_strategy_distance],
    "Store Farthest, Restore Closest": [store_farthest_strategy_distance],
    "Store Random, Restore Closest": [store_random_strategy_distance],
    "Store Random, Restore Random": [choose_random_strategy_distance],
}, columns=["Distance Traveled"], orient="index").sort_values(by="Distance Traveled")

,Distance Traveled
Policy Iteration,228.00
Value Iteration,228.00
"Store Closest, Restore Closest",228.00
"Store Random, Restore Closest",253.92
"Store Random, Restore Random",259.42
"Store Farthest, Restore Closest",274.00


The policy generated using PolicyIteration and also the one generated using ValueIteration is as good as the Closest-Field-Strategy and outperforms all other strategies.